In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

tf.random.set_seed(42)

In [ ]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

Here is a simple example of how to work with the nnlm-en-dim50 module:

In [ ]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [ ]:
embeddings

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.16589954,  0.0254965 ,  0.1574857 ,  0.17688066,  0.02911299,
        -0.03092718,  0.19445257, -0.05709129, -0.08631689, -0.04391516,
         0.13032274,  0.10905275, -0.08515751,  0.01056632, -0.17220995,
        -0.17925954,  0.19556305,  0.0802278 , -0.03247919, -0.49176937,
        -0.07767699, -0.03160921, -0.13952136,  0.05959712,  0.06858718,
         0.22386682, -0.16653948,  0.19412343, -0.05491862,  0.10997339,
        -0.15811177, -0.02576607, -0.07910853, -0.258499  , -0.04206644,
        -0.20052543,  0.1705603 , -0.15314153,  0.0039225 , -0.28694248,
         0.02468278,  0.11069503,  0.03733957,  0.01433943, -0.11048374,
         0.11931834, -0.11552787, -0.11110869,  0.02384969, -0.07074881],
       [ 0.1437864 ,  0.08291595,  0.10897306,  0.04464385, -0.03630389,
        -0.12605834,  0.20263346,  0.12862863, -0.07873426, -0.01195358,
         0.0020956 , -0.03080653, -0.08019945, -0.18797135, -0.11973457,
 

Now let's implement this in our Sentiment Analysis model:

The `hub.KerasLayer` layer downloads the module from the given URL. This particular module is a sentence encoder: it takes strings as input and encodes each one as a single vector (in this case, a 50-dimensional vector).
We can then add two simple Dense layers to create a good sentiment analysis model.
By default, a `hub.KerasLayer` is not trainable, but you can set `trainable=True` when creating it to change that so that you can fine-tune it for your task.

In [ ]:
model = keras.Sequential([
    # The last part of the URL specified that we wanted version 1 of the model.
    # This versioning ensures that if a new module version is released, it will not break our model.
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [ ]:
for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKJ11LK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKJ11LK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKJ11LK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
test_set = datasets["test"]
datasets["test"].num_examples

AttributeError: ignored

In [ ]:
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=10)

Epoch 1/10
781/781 [==============================] - 5s 6ms/step - loss: 0.5460 - accuracy: 0.7267
Epoch 2/10
781/781 [==============================] - 4s 5ms/step - loss: 0.5129 - accuracy: 0.7494
Epoch 3/10
781/781 [==============================] - 4s 5ms/step - loss: 0.5082 - accuracy: 0.7530
Epoch 4/10
781/781 [==============================] - 4s 6ms/step - loss: 0.5046 - accuracy: 0.7538
Epoch 5/10
781/781 [==============================] - 4s 6ms/step - loss: 0.5017 - accuracy: 0.7561
Epoch 6/10
781/781 [==============================] - 4s 5ms/step - loss: 0.4990 - accuracy: 0.7585
Epoch 7/10
781/781 [==============================] - 4s 5ms/step - loss: 0.4969 - accuracy: 0.7596
Epoch 8/10
781/781 [==============================] - 4s 6ms/step - loss: 0.4944 - accuracy: 0.7608
Epoch 9/10
781/781 [==============================] - 5s 6ms/step - loss: 0.4923 - accuracy: 0.7625
Epoch 10/10
781/781 [==============================] - 5s 6ms/step - loss: 0.4899 - accuracy: 0.7648

In [ ]:
test_set.prefetch(2)

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [ ]:
model.evaluate(test_set.batch(25000))

1/1 [==============================] - 0s 2ms/step - loss: 0.5071 - accuracy: 0.7508


[0.5070638656616211, 0.7508400082588196]

In [ ]:
np.save('/content/pretrained_embedding.npy', history.history)
model.save("/content/pretrained_embedding.h5")

In [ ]:
vocab_size = 100
embed_size = 10

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

# We set return_state=True when creating the LSTM layer so that  we can get its final
# hidden state and pass it to the decoder.
encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

# TrainingSampler() tell the decoder at each step what it should pretend the previous output was.
# During inference, this should be the embedding of the token that was actually output.
# During training, it should be the embedding of the previous target token.
# This is why we used the TrainingSampler()
sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

Since we are using an LSTM cell, it actually returns two hidden states (short term and long term).

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)